In [1]:
%pip install accelerate bitsandbytes langchain faiss-gpu dspy pypdf sentence-transformers

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [3]:
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv())

True

In [4]:
from huggingface_hub import login
login(os.environ["HF_TOKEN"])

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\chris\.cache\huggingface\token
Login successful


In [5]:
from pprint import pprint
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer, AutoConfig

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

# torch.set_default_device('cuda')

In [6]:
model_name = "google/gemma-2b-it"

model_config = AutoConfig.from_pretrained(model_name)
model_config.hidden_act = 'gelu_pytorch_tanh'
model_config.hidden_activation = 'gelu_pytorch_tanh'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, config=model_config, device_map='auto', quantization_config=nf4_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
streamer = TextStreamer(tokenizer)

In [8]:
x = tokenizer('example texts', padding=True, return_tensors='pt', )
x

{'input_ids': tensor([[    2,  7310, 26448]]), 'attention_mask': tensor([[1, 1, 1]])}

In [9]:
from pathlib import Path
from itertools import chain
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [10]:
pdfs = list(Path('./data').absolute().iterdir())
pdfs

[WindowsPath('c:/Users/chris/repos/ml/notebooks/data/4 Skills and qualities table.pdf'),
 WindowsPath('c:/Users/chris/repos/ml/notebooks/data/CS_Behaviours_2018.pdf'),
 WindowsPath('c:/Users/chris/repos/ml/notebooks/data/Sophia+CV+2021.pdf')]

In [11]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceBgeEmbeddings
model_name = "BAAI/bge-large-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

embedding_model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    query_instruction="Represent this sentence for searching relevant passages: ",
)

In [12]:
f"{pdfs[0]}"

'c:\\Users\\chris\\repos\\ml\\notebooks\\data\\4 Skills and qualities table.pdf'

In [33]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

pages = [PyPDFLoader(f"{pdf}").load_and_split(text_splitter) for pdf in pdfs]

In [34]:
faiss_index = FAISS.from_documents(pages[0], embedding_model)

for page in pages[1:]:
    faiss_index.merge_from(FAISS.from_documents(page, embedding_model))

In [36]:
docs = faiss_index.similarity_search("What places can I use to find job opportunities", k=4)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content)

2: • Role-play
• Oral briefing
• Assessment centre. 
 
Depending on the type of assessment, for 
example in an application form or at an 
interview, you may be asked to give examples 
of when you have demonstrated a particular 
behaviour.This might be at work or somewhere else 
such as work experience, volunteering or in 
connection with a hobby.
Alternatively, you may be asked how you would 
behave in a given situation, for example by 
using Situational Judgement Tests or through 
situational interview questions. You could 
also be asked to demonstrate a behaviour 
in real-time, for example during a structured 
behavioural assessment.
Your behaviours may be assessed alongside 
other elements of the Success Profile to get 
a more rounded picture of your suitability for 
the role. The job description will outline the 
elements required for the role and the selection 
method(s) that will be used. 
The Civil Service is a diverse and inclusive 
workplace and we want to help you
2: The Civi

In [14]:
from transformers import pipeline

pipe = pipeline(
    'text-generation', model=model, tokenizer=tokenizer, max_new_tokens=512, temperature=0, do_sample=False,)

In [15]:
from langchain import HuggingFacePipeline
from langchain.retrievers.multi_query import MultiQueryRetriever
llm = HuggingFacePipeline(pipeline=pipe)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=faiss_index.as_retriever(), llm=llm
)

In [16]:
tokenizer.apply_chat_template([{ "role": "user", "content": "Write a hello world program" }], tokenize=False)

'<bos><start_of_turn>user\nWrite a hello world program<end_of_turn>\n'

In [48]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter


prompt = ChatPromptTemplate.from_template(
    "<bos><start_of_turn>user\n"
    "Using either your own expert knowledge (no tag), enriched with the following contextual data (denoted by CONTEXT), "
    "please answer my prompt (denoted by PROMPT)\nIf there you cannot answer from the context alone, please let "
    "the user know, but still answer with your expert knowledge.\n"
    "CONTEXT\n{context}\nPROMPT\n{prompt}<end_of_turn>\n<start_of_turn>model\n")


In [49]:
faiss_index.similarity_search('what is the key contribution of llama2')

[Document(page_content='PROFILE  Tin dicatas con Ita, caus ex mil us ceresil larisque crunimpliam\nACCOUNT EXECUTIVE KANDA CONSULTING   / JUNE 2021 – PRESENT • Produced political research and reports to support the redevelopment schemes of a number of high-profile clients in the Central London development industry and local authorities.  • Gathered feedback on a wide range of regeneration projects in Westminster and The City of London and produced detailed reports including quantitative and qualitative analysis on this feedback to the client.  • Social Media and website content management – delivered high quality content for bespoke consultation websites and newsletters as well as the Company’s LinkedIn feed. • Stakeholder engagement – communicated and maintained positive relationships with a wide range of community, political and commercial stakeholders.  • Delivered research to support Directors in securing new business.', metadata={'source': 'C:\\Users\\chris\\repos\\ml\\notebooks\\

In [76]:
# Using the newer chains api
from langchain_core.runnables import RunnableGenerator
from typing import Iterable
from langchain.retrievers.document_compressors import EmbeddingsFilter


def parse(message):
    return message.split("<start_of_turn>model\n")

def ret_func(query_input):
    docs = faiss_index.max_marginal_relevance_search(query_input['prompt'], k=7, fetch_k=20, lambda_mult=0.4)
    # docs = retriever_from_llm.invoke(query_input['prompt'])

    print('\n\n')
    doc_name = doc.metadata["source"].split("\\")[-1]
    contexts = [f'Source: {doc_name} - Data: {doc.metadata["page"]}: {doc.page_content}' for doc in docs]
    return '\n'.join(contexts)

# print(ret_func({'prompt':'what are the key contributions of llama2'}))

In [78]:
chain = (
    {
        # "context": retriever_from_llm,
        "context": ret_func,
        "prompt": itemgetter('prompt'),
    }
    | prompt
    | llm
    | parse

)

output = chain.invoke({"prompt": "Summarise my work experience from my CV and give a list of potential future opportunities with explanations of why they are suitable and industries they can be applied in"})

C:\Users\chris\miniconda3\envs\ai\Lib\site-packages\transformers\generation\configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [79]:
# print(output[0])

In [80]:
print(output[-1])

## Summary of Work Experience

Based on the context, I have gained experience in various roles, demonstrating my ability to adapt to different situations and contribute to achieving organizational goals. My key responsibilities include:

* **Project Management:** Planning, organizing, and executing projects to deliver successful outcomes.
* **Communication:** Engaging in clear and concise communication with stakeholders, both verbally and in writing.
* **Teamwork:** Collaborating effectively with colleagues from diverse backgrounds and fostering a positive team environment.
* **Problem-solving:** Identifying and resolving challenges through creative and innovative solutions.
* **Strategic Thinking:** Developing and implementing strategic plans that align with organizational goals.
* **Leadership:** Inspiring and motivating team members to achieve their full potential.

## Potential Future Opportunities

Based on my skills and experience, I am suitable for the following potential future

In [57]:
print(chain.invoke({'prompt': 'What are approaches to deal with problems where the number of features is greater than the number of samples?'})[-1])

C:\Users\chris\miniconda3\envs\ai\Lib\site-packages\transformers\generation\configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Sure, here are approaches to deal with problems where the number of features is greater than the number of samples:

- **Resampling methods:** Resampling methods involve repeatedly drawing samples from the training set and refitting the model on each sample. This approach can help to reduce the overfitting of the model.
- **Tree pruning:** Tree pruning is a technique that can be used to reduce the complexity of a tree-based model. This approach can help to improve the performance of the model on the test set.
- **Feature selection:** Feature selection is a technique that can be used to identify the most important features in a dataset. This approach can help to improve the performance of the model on the test set.


In [65]:
retriever_from_llm._get_relevant_documents('What are approaches to deal with problems where the number of features is greater than the number of samples?')

TypeError: MultiQueryRetriever._get_relevant_documents() missing 1 required keyword-only argument: 'run_manager'

In [64]:
import inspect
print(inspect.getsource(MultiQueryRetriever))

class MultiQueryRetriever(BaseRetriever):
    """Given a query, use an LLM to write a set of queries.

    Retrieve docs for each query. Return the unique union of all retrieved docs.
    """

    retriever: BaseRetriever
    llm_chain: LLMChain
    verbose: bool = True
    parser_key: str = "lines"
    """DEPRECATED. parser_key is no longer used and should not be specified."""
    include_original: bool = False
    """Whether to include the original query in the list of generated queries."""

    @classmethod
    def from_llm(
        cls,
        retriever: BaseRetriever,
        llm: BaseLanguageModel,
        prompt: PromptTemplate = DEFAULT_QUERY_PROMPT,
        parser_key: Optional[str] = None,
        include_original: bool = False,
    ) -> "MultiQueryRetriever":
        """Initialize from llm using default template.

        Args:
            retriever: retriever to query documents from
            llm: llm for query generation using DEFAULT_QUERY_PROMPT
            include_or